### Practice: Parameter Efficient Fine-Tuning
In this notebook, you're gonna fine-tune large language models within limited GPU memory.

Prompt Tuning

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

import transformers
from tqdm.auto import tqdm, trange
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "Qwen/Qwen2.5-0.5B"

# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B")
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B",token = "", local_files_only = True)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B",token = "", local_files_only = True)

In [ ]:
model.device

In [ ]:

prompt = "I am happy"
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=30
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

Prompt Tuning

In [ ]:

from transformers import TrainingArguments, Trainer
from peft import get_peft_model, PromptTuningConfig, TaskType



# Configure PEFT for Prompt Tuning
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,  # Prompt tuning for causal language modeling
    num_virtual_tokens=10,         # Define 10 tunable soft prompt tokens
    tokenizer_name_or_path=model_name
)

# Wrap the model with PEFT
model = get_peft_model(model, peft_config)

# Print trainable parameters
model.print_trainable_parameters()


In [ ]:

# Create training data: User input → Negated response
training_examples = [
    {"input": "The sky is blue.", "output": "The sky is not blue."},
    {"input": "Water is essential for life.", "output": "Water is not essential for life."},
    {"input": "I love programming.", "output": "I do not love programming."},
    {"input": "This AI model is powerful.", "output": "This AI model is not powerful."},
]

# Create training data: User input → Semantically negated response
training_examples = [
    {"input": "The sky is blue.", "output": "The sky is gray and cloudy."},
    {"input": "Water is essential for life.", "output": "Water is not always necessary for survival."},
    {"input": "I love programming.", "output": "I struggle with programming."},
    {"input": "This AI model is powerful.", "output": "This AI model lacks efficiency."},
    {"input": "Dogs are friendly animals.", "output": "Dogs can sometimes be aggressive."},
    {"input": "Summer is the hottest season.", "output": "Winter is far colder than summer."},
    {"input": "Reading books is a great hobby.", "output": "Reading books can be tedious for some."},
    {"input": "Paris is the capital of France.", "output": "Paris is just one of many important cities in France."},
    {"input": "This phone is expensive.", "output": "This phone is quite affordable."},
    {"input": "The Earth revolves around the Sun.", "output": "The Sun does not revolve around the Earth."},
    {"input": "Math is an interesting subject.", "output": "Math can be boring for some students."},
    {"input": "Ice melts in the heat.", "output": "Ice can remain solid in extreme conditions."},
    {"input": "Chocolate is sweet.", "output": "Chocolate can sometimes taste bitter."},
    {"input": "Running is good for health.", "output": "Running excessively can be harmful to health."},
    {"input": "Music makes me happy.", "output": "Music sometimes makes me feel nostalgic."},
    {"input": "This game is fun.", "output": "This game can become repetitive over time."},
    {"input": "Apples are delicious.", "output": "Some people find apples too sour."},
    {"input": "She is a great dancer.", "output": "Her dancing style is not to everyone's taste."},
    {"input": "Coffee keeps me awake.", "output": "Coffee does not always work to keep me awake."},
    {"input": "Birds can fly.", "output": "Some birds, like penguins, cannot fly."},
    {"input": "The internet is fast here.", "output": "The internet can be slow at times."},
    {"input": "Traveling is exciting.", "output": "Traveling can be exhausting."},
    {"input": "This book is well-written.", "output": "This book is overly complex."},
    {"input": "He is a kind person.", "output": "He can be indifferent at times."},
    {"input": "Fresh fruit is healthy.", "output": "Too much fruit can be bad for digestion."},
    {"input": "Learning a new language is useful.", "output": "Learning a new language takes a lot of effort."},
    {"input": "Exercising daily is beneficial.", "output": "Exercising too much can cause injuries."},
    {"input": "Diamonds are valuable.", "output": "Diamonds are overpriced."},
    {"input": "This restaurant serves delicious food.", "output": "The food here is quite average."},
    {"input": "He speaks English fluently.", "output": "His English has some noticeable mistakes."},
    {"input": "Her voice is beautiful.", "output": "Her voice lacks depth."},
    {"input": "This software is easy to use.", "output": "This software can be confusing at first."},
    {"input": "The sunset is stunning.", "output": "The sunset looks dull on cloudy days."},
    {"input": "I enjoy watching movies.", "output": "Movies often fail to keep my attention."},
    {"input": "He is an honest man.", "output": "He sometimes bends the truth."},
    {"input": "This dress fits perfectly.", "output": "This dress feels a little tight."},
    {"input": "The water is crystal clear.", "output": "The water has a slight murkiness."},
    {"input": "She is always on time.", "output": "She occasionally arrives late."},
    {"input": "This tool is reliable.", "output": "This tool can malfunction at times."},
    {"input": "The party was lively.", "output": "The party felt a little dull."},
    {"input": "He is a generous person.", "output": "He can be a bit reserved with his money."},
    {"input": "The road is smooth.", "output": "The road has some rough patches."},
    {"input": "Learning history is fascinating.", "output": "History can be dry for some students."},
    {"input": "She is very patient.", "output": "She gets impatient in stressful situations."},
    {"input": "This new policy is fair.", "output": "This new policy benefits some more than others."},
    {"input": "That was a smart decision.", "output": "That decision had its flaws."},
    {"input": "The music was energetic.", "output": "The music was a bit too loud for my taste."},
    {"input": "The cake is delicious.", "output": "The cake is slightly too sweet."},
]



# Tokenize data for training
train_data = []
for example in training_examples:
    messages = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": example["input"]},
        {"role": "assistant", "content": example["output"]+" <eos>"}
    ]

#     messages = [
#     {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
#     {"role": "user", "content": prompt}
# ]
    
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    inputs["labels"] = inputs["input_ids"].clone()  # Labels are the same as input_ids for causal LM training
    train_data.append(inputs)

# Collate function for DataLoader
def collate_fn(batch):
    input_ids = torch.nn.utils.rnn.pad_sequence([b["input_ids"].squeeze(0) for b in batch], batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = torch.nn.utils.rnn.pad_sequence([b["attention_mask"].squeeze(0) for b in batch], batch_first=True, padding_value=0)
    labels = torch.nn.utils.rnn.pad_sequence([b["labels"].squeeze(0) for b in batch], batch_first=True, padding_value=-100)
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

# Define Trainer arguments
training_args = TrainingArguments(
    per_device_train_batch_size=3,
    num_train_epochs=100,
    logging_steps=10,
    output_dir="./negation_tuned_model",
    save_strategy="epoch",
    evaluation_strategy="no",
    learning_rate=5e-4
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    data_collator=collate_fn
)

# Train the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./negation_tuned_model")

print("Training complete! The negation-tuned model is saved.")

# --- TEST THE MODEL WITH A NEW INPUT ---
def negate_statement(user_input):

    prompt = user_input
    messages = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # messages = [
    #     {"role": "system", "content": "Your task is to negate any user statement."},
    #     {"role": "user", "content": user_input}
    # ]
    
    # # text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    # text = user_input
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=50
    )
    generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return generated_text


In [ ]:

# Example test case
user_input = "I am happy."
negated_response = negate_statement(user_input)
print(f"User: {user_input}")
print(f"Model: {negated_response}")
